### Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
from tqdm  import tqdm
from statsmodels.discrete.discrete_model import Logit
import statsmodels.api as sm
from scipy.optimize import minimize
import sys

## Problem 1: Dynamic Optimization 

In [2]:
data = pd.read_csv(r"C:\Users\Popov\Documents\Studies\NES_studies\Python\EIO\HA2\RUST_DATA.csv")

In [3]:
data.head(4)

,0,level_1,state,decision
0,4403,1983-05-01,0.0,0.0
1,4403,1983-06-01,0.0,0.0
2,4403,1983-07-01,1.0,0.0
3,4403,1983-08-01,2.0,0.0


### Task 1: Static model

In [4]:
# select data
X = data['state']
X = sm.add_constant(X)
Y = data['decision'].astype(int)

# fit logit regression
logit = Logit(Y, X).fit(cov_type="hc2")
print(logit.summary())

Optimization terminated successfully.
         Current function value: 0.036576
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:               decision   No. Observations:                 8260
Model:                          Logit   Df Residuals:                     8258
Method:                           MLE   Df Model:                            1
Date:                Fri, 18 Oct 2024   Pseudo R-squ.:                  0.1377
Time:                        12:00:32   Log-Likelihood:                -302.12
converged:                       True   LL-Null:                       -350.35
Covariance Type:                  hc2   LLR p-value:                 9.139e-23
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.3942      0.285    -25.917      0.000      -7.953      -6.835
state          0.0712      0

In [5]:
coefs = logit.params
theta_1_static = coefs[1] 
RC_static = coefs[0]
print(f"Estimates: theta_1_static = {theta_1_static:.3f}, RC_static = {RC_static:.3f}")

Estimates: theta_1_static = 0.071, RC_static = -7.394


In [6]:
import statsmodels.api as sm

# Assuming 'logit' is your fitted Logit model
logit_results = logit.summary2()  # Use summary2 for a more compact table

# Extract the table with the regression results (coefficients, SE, etc.)
regression_table = logit_results.tables[1]

# Convert to LaTeX format with 3 decimal places using float_format
latex_output = regression_table.to_latex(float_format="%.3f")

# Save to a .tex file
with open("1_1.tex", "w") as f:
    f.write(latex_output)

### Task 2: Dynamic model

#### Transitions probabilities

In [7]:
# define difference variable - changes in states (mileage groups)
data['state_discrete'] = data['state'] - data['state'].shift(1)

# define the total sums of the changes (transitions)
N = data['state_discrete'].notna().sum().sum()

# finally, define the number of transitions into particular states
t1 = (data['state_discrete'] == 0).sum().sum() / N
t2 = (data['state_discrete'] == 1).sum().sum() / N
t3 = (data['state_discrete'] == 2).sum().sum() / N

# print them
rounded_probs = (round(t1, 3), round(t2, 3), round(t3, 3))
print(rounded_probs)

(0.343, 0.626, 0.011)


#### Transition matrix

In [8]:
# determine the number of states
T = max(data['state'].astype(int))+1 # add one, as python won't see the last value

# crete an empty transition matrix
P = np.zeros((T, T))

# add probabilities of staying in the same state
np.fill_diagonal(P, t1)

# add probability of getting into the next state
P = P + np.diag(np.ones(T - 1) * t2, 1)

# and probability of getting 2 states ahead
P = P + np.diag(np.ones(T - 2) * t3, 2)

# adjust the last column to ensure row sums equal 1
P[:, -1] += 1 - P.sum(1)

# see the matrix
P_rounded = np.round(P, 3)
print(P_rounded)

[[0.343 0.626 0.011 ... 0.    0.    0.019]
 [0.    0.343 0.626 ... 0.    0.    0.019]
 [0.    0.    0.343 ... 0.    0.    0.019]
 ...
 [0.    0.    0.    ... 0.343 0.626 0.031]
 [0.    0.    0.    ... 0.    0.343 0.657]
 [0.    0.    0.    ... 0.    0.    1.   ]]


#### Optimal value function

Define special parameters

In [9]:
beta = 0.8 # recommendation from the article - for quicker convergence
gamma = 0.5772156649 # Euler-Mascheroni constant

# set grid for state x
x_linspace = np.arange(T, dtype=np.float64)

Define utility function

In [10]:
# utility function
def u(x, d, theta, RC):
    if d == 0:
        return -theta * x
    if d == 1:
        return -RC

Set value function

In [11]:
# Value function 
def V_it(V, theta, RC):
    EV = P @ V # expected value = transition matrix*value function
    replace = u(0, 1, theta, RC) + beta * EV[0] # replace option = utility + discount factor
    maintain = u(x_linspace, 0, theta, RC) + beta * EV # maintain option utility + discount factor
    V_new = gamma + np.log(np.exp(maintain) + np.exp(replace)) # new value function Vt+1
    # note: EV[0] - for replace option we apply only the discount for the next
    # period, while for maintain - for the whole vector of future values
    return V_new

#### Maximum likelihhod

Utilize Fixed Point algorithm for the Bellman equation (note : $\theta_1$, $RC$ are fixed)

In [12]:
# Fixed point of BE search
def solve_FP_Bellman(theta, RC):
    tol=1e-8 # set tolerance of convergence stopping rule
    maxiter=1000000 # maximum number of iterations
    V = np.ones(T) # starting parameters - default guessed value fucntion
    # start iterating
    i = 0 # iteration counter
    dist = 1000 # difference between consecutive value function estimates - technical
    for i in range(maxiter): # add timer as well
        V_new = V_it(V, theta, RC) # update value function
        distance = np.linalg.norm(V - V_new) # distance Between Current and New Value Functions
        if distance < tol: # if it is lower than tolerance - stop (convergence check)
            return V_new
        else: # if not - continue the cycle
            i += 1 # add 1 to iteration counter
            V = V_new.copy() # redefine V as the last computed V
    return V

With the optimal value function found - move to Maximum likelihood to find optimal $\theta_1$, $RC$

In [13]:
def log_likelihood(param, dataset):
    theta, RC = param # unpack parameters
    # Apply FP approach, find optimal value function (solve the Bellman equation)
    V = solve_FP_Bellman(theta, RC)
    # set log-likelihood cumulative variable
    logL = 0
    # compute log-likelihood
    for i in range(dataset.shape[0]):
        decision = dataset['decision'].values[i] # extract decision (maintain/replace)
        state = int(dataset['state'].values[i]) # extract state (current mileage group)
        # calculate discounted utility for both decisions
        maintain_u = u(state, 0, theta, RC) + beta * V[state]
        replace_u = u(0, 1, theta, RC) + beta * V[0]
        # compute probabilities (via logistic distribution, of course)
        p_maintain = 1 / (1 + np.exp(replace_u - maintain_u))
        p_replace = 1 - p_maintain
        # update the Log-Likelihood with the probability of chosen decision
        if decision == 0:
            logL += np.log(p_maintain)
        if decision == 1:
            logL += np.log(p_replace)
    return -logL # return the negative Log-likelihood (because we have a minimizating solver by default

Finally, time to get the estimated values

#### Estimates

In [14]:
def estimates_bus(dataset):
    x_start = [theta_1_static,RC_static] # choose staring parameters - use ones from the static model
    return minimize(log_likelihood, x0=x_start, args=(dataset), bounds=((0, None), (0, None))).x
    # minimize() - minimization solver
    # log_likelihood - the objective function that minimize() will try to minimize
    # [theta_1_static,RC_static] - starting point for the optimization
    # args=(dataset) - extra parameters, used for the objective function
    # bounds=((0, None), (0, None)) - the parameters are constrained to be non-negative
    # ().x - extracts the optimized parameter values

In [15]:
theta_dynamic, RC_dynamic = estimates_bus(data)
print(f"Estimates: theta_1_dynamic = {theta_dynamic:.3f}, RC_dynamic = {RC_dynamic:.3f}")

Estimates: theta_1_dynamic = 0.017, RC_dynamic = 7.587


#### Bootstrap

In [16]:
%%time
# create arrays to store bootstrap results
theta_obs = np.zeros(1000)
RC_obs = np.zeros(1000)

for i in tqdm(range(1000)):
    Bootstrap_sample = data.iloc[np.random.choice(data.shape[0], data.shape[0])]
    # np.random.choice(data.shape[0], data.shape[0]) - generates a list of random indices,
    # with replacement, of the same length as the original dataset
    # data.iloc[] - extracts them to the new bootstrap sample
    theta_b, RC_b = estimates_bus(Bootstrap_sample) # estimates theta_dynamic, RC_dynamic
    # now store them
    theta_obs[i] = theta_b
    RC_obs[i] = RC_b

100%|██████████| 1000/1000 [1:06:20<00:00,  3.98s/it]

CPU times: total: 48min 17s
Wall time: 1h 6min 20s


#### SE estimation

(based on Bootstraped empirical distribution)

In [17]:
theta_std = np.sqrt(np.square(np.array(theta_obs - theta_dynamic)).sum() / (len(theta_obs) - 1))
RC_std = np.sqrt(np.square(np.array(RC_obs - RC_dynamic)).sum() / (len(RC_obs) - 1))
print(f"Estimates: theta_1_dynamic_std = {theta_std:.3f}, RC_dynamic_std = {RC_std:.3f}")

Estimates: theta_1_dynamic_std = 0.001, RC_dynamic_std = 0.311


In [18]:
table_dynamic = {
    'Parameter': ['theta_1_dynamic', 'RC_dynamic'],
    'Estimate': [theta_dynamic, RC_dynamic],
    'Std Error': [theta_std, RC_std]
}

dynamic_table = pd.DataFrame(table_dynamic)
dynamic_table = dynamic_table.round(3)

# Display the table
print(dynamic_table)

# Export LaTeX
latex_code = dynamic_table.to_latex(index=False, caption="Dynamic Parameter Estimates and Bootstrapped Standard Errors")

# Save the LaTeX table to a .tex file
with open("dynamic_table.tex", "w") as f:
    f.write(latex_code)

         Parameter  Estimate  Std Error
0  theta_1_dynamic     0.017      0.001
1       RC_dynamic     7.587      0.311


## Problem 2: 

### Task 1: Incomplete information

In [19]:
data2 = pd.read_stata(r"C:\Users\Popov\Documents\Studies\NES_studies\Python\EIO\HA2\drinking_in_family_RLMS_mod.dta")

In [20]:
data2.head(4) # note that to open this dataset correctly couple of operation were conducted in stata

,HH_ID,educ_husband,age1_husband,weight1_husband,height1_husband,drink1_husband,educ_wife,age2_wife,weight2_wife,height2_wife,drink2_wife
0,1,16.0,23,76.0,185.0,1,18.0,21,45.0,164.0,1
1,2,16.0,43,85.0,174.0,1,18.0,48,53.0,162.0,1
2,3,21.0,29,80.0,187.0,1,16.0,24,56.0,158.0,1
3,4,14.0,37,57.0,167.0,1,9.0,40,60.0,166.0,1


Let's create separate datasets for husband/wife

In [21]:
# husband dataset
males = data2.copy()
males.rename(columns={'educ_husband':'educ_i','age1_husband':'age_i','weight1_husband':'weight_i',
                         'height1_husband':'height_i','drink1_husband':'drink_i'}, inplace=True)
# educ_i - education of a husband (i) in the husband dataset
# educ_not_i - education of a wife (not_i) in the husband dataset 
males.rename(columns={'educ_wife':'educ_not_i','age2_wife':'age_not_i','weight2_wife':'weight_not_i',
                         'height2_wife':'height_not_i','drink2_wife':'drink_not_i'}, inplace=True)
males['spouse'] = 'husband'

# wife dataset
females = data2.copy()
females.rename(columns={'educ_wife':'educ_i','age2_wife':'age_i','weight2_wife':'weight_i',
                      'height2_wife':'height_i','drink2_wife':'drink_i'}, inplace=True)
# educ_h_i - education of a wife (i) in the wife dataset
# educ_w_not_i - education of a husband (not_i) in the wife dataset 

females.rename(columns={'educ_husband':'educ_not_i','age1_husband':'age_not_i','weight1_husband':'weight_not_i',
                      'height1_husband':'height_not_i','drink1_husband':'drink_not_i'}, inplace=True)
females['spouse'] = 'wife'

In [22]:
data_RLMS = pd.concat([males, females], ignore_index=True,axis=0) # axis=0 - rows are stacked on top pof each other
data_RLMS = data_RLMS.iloc[:, [11] + list(range(0, 11))]

In [23]:
data_RLMS.sample(4)

,spouse,HH_ID,educ_i,age_i,weight_i,height_i,drink_i,educ_not_i,age_not_i,weight_not_i,height_not_i,drink_not_i
27175,wife,24005,19.0,22,70.0,162.0,1,14.0,24,75.0,178.0,1
8348,husband,14725,8.0,36,80.0,175.0,0,17.0,38,85.0,160.0,0
22175,wife,14489,16.0,33,48.0,158.0,1,8.0,34,60.0,167.0,1
22522,wife,15102,18.0,34,95.0,167.0,1,8.0,37,80.0,176.0,1


#### First stage: beliefs

In [24]:
Y = data_RLMS['drink_i']
X = data_RLMS.drop(['spouse', 'HH_ID', 'drink_i', 'drink_not_i'],axis=1) # remove unrelevant variables
X = sm.add_constant(X)

# run our beloved logit
logit = Logit(Y, X).fit(cov_type="hc2")
print(logit.summary())

Optimization terminated successfully.
         Current function value: 0.516084
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                drink_i   No. Observations:                27890
Model:                          Logit   Df Residuals:                    27881
Method:                           MLE   Df Model:                            8
Date:                Fri, 18 Oct 2024   Pseudo R-squ.:                 0.02416
Time:                        13:06:57   Log-Likelihood:                -14394.
converged:                       True   LL-Null:                       -14750.
Covariance Type:                  hc2   LLR p-value:                1.295e-148
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            5.0037      0.529      9.459      0.000       3.967       6.040
educ_i          -0.0237

In [25]:
import statsmodels.api as sm

logit_results = logit.summary2()  # Use summary2 for a more compact table

# Extract the table with the regression results
regression_table = logit_results.tables[1]

# Convert to LaTeX format with 3 decimal
latex_output = regression_table.to_latex(float_format="%.3f")

# Save
with open("2_1_a.tex", "w") as f:
    f.write(latex_output)

Now we can get estimates of drink_i and drink_not_i - predicted by our logit regression

In [26]:
# idea - to assign drink_pred for wife to husband observation (and vice versa) - these will be beliefs of a husband (wife)
data_RLMS['drink_pred'] = logit.predict(X) # prediction for the whole dataset
drink_pred_wifes = np.array(data_RLMS.loc[data_RLMS['spouse']=='wife','drink_pred']) # for wifes
drink_pred_husbands = np.array(data_RLMS.loc[data_RLMS['spouse']=='husband','drink_pred']) # for husbands
data_RLMS['belief']=np.concatenate([drink_pred_wifes,drink_pred_husbands]) # now unite - note that the dataset itself has husband obs. first
data_RLMS = data_RLMS.drop(['drink_pred'],axis = 1) # drop the initial column

In [27]:
data_RLMS.sample(6)

,spouse,HH_ID,educ_i,age_i,weight_i,height_i,drink_i,educ_not_i,age_not_i,weight_not_i,height_not_i,drink_not_i,belief
4597,husband,8008,16.0,41,48.0,172.0,1,16.0,41,120.0,165.0,1,0.838733
24769,wife,19214,18.0,33,70.0,154.0,0,16.0,36,82.0,167.0,0,0.860388
26428,wife,22519,18.0,20,87.0,180.0,0,16.0,23,64.0,163.0,0,0.672296
3383,husband,5939,21.0,46,88.0,176.0,1,21.0,46,76.0,160.0,0,0.790152
25131,wife,19887,23.0,35,52.0,160.0,1,21.0,42,85.0,179.0,1,0.865391
10101,husband,17954,16.0,38,74.0,166.0,1,18.0,36,83.0,160.0,1,0.804775


#### Second stage: utility optimization

Using the estimated $\widehat{I\{\text{drink}_{-i}\}}$ for each spouse, run the second stage logit.

In [28]:
Y2 = data_RLMS['drink_i']
X2= data_RLMS[['educ_i', 'age_i', 'weight_i', 'height_i', 'belief']]
X2 = sm.add_constant(X2)

logit2 = Logit(Y2, X2).fit(cov_type="hc2")
print(logit2.summary())

Optimization terminated successfully.
         Current function value: 0.520962
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                drink_i   No. Observations:                27890
Model:                          Logit   Df Residuals:                    27884
Method:                           MLE   Df Model:                            5
Date:                Fri, 18 Oct 2024   Pseudo R-squ.:                 0.01494
Time:                        13:06:58   Log-Likelihood:                -14530.
converged:                       True   LL-Null:                       -14750.
Covariance Type:                  hc2   LLR p-value:                 5.195e-93
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4473      0.720      2.010      0.044       0.036       2.859
educ_i        -0.0371      0.

In [29]:
import statsmodels.api as sm

logit_results_2 = logit2.summary2()  # Use summary2 for a more compact table

# Extract the table with the regression results
regression_table_2 = logit_results_2.tables[1]

# Convert to LaTeX format with 3 decimal
latex_output_2 = regression_table_2.to_latex(float_format="%.3f")

# Save
with open("2_1_b.tex", "w") as f:
    f.write(latex_output_2)

### Task 2: Complete information

Define logit choice probability function (Lamdba) 

In [30]:
def logit_choic_pr(x):
    return np.exp(x)/(1+np.exp(x))

In [31]:
data2.columns

Index(['HH_ID', 'educ_husband', 'age1_husband', 'weight1_husband',
       'height1_husband', 'drink1_husband', 'educ_wife', 'age2_wife',
       'weight2_wife', 'height2_wife', 'drink2_wife'],
      dtype='object')

#### Negative peer effects case

In [32]:
def MaxL_negative(param):
    gamma_1, gamma_2, beta_1, beta_2 ,beta_3, beta_4, beta_5, beta_6, beta_7, beta_8 = param
    logL = 0 # as before - camulative log-likelihood
    for i in range(data2.shape[0]): # working with the initial dataset
        # define outcome variables
        d_1 = data2['drink1_husband'][i]
        d_2 = data2['drink2_wife'][i]
        
        # define covariates
        # age
        X_1a = data2['age1_husband'][i]
        X_2a = data2['age2_wife'][i]
        #educ
        X_1e = data2['educ_husband'][i]
        X_2e = data2['educ_wife'][i]
        # weight
        X_1w = data2['weight1_husband'][i]
        X_2w = data2['weight2_wife'][i]
        # height
        X_1h = data2['height1_husband'][i]
        X_2h = data2['height2_wife'][i]

        # define probabilities
        P_00 = logit_choic_pr(-beta_1 * X_1a - beta_3*X_1e - beta_5*X_1w - beta_7*X_1h) * \
        logit_choic_pr(-beta_2 * X_2a - beta_4*X_2e - beta_6*X_2w - beta_8*X_2h)
        
        P_11 = (1-logit_choic_pr(-beta_1 * X_1a - beta_3*X_1e - beta_5*X_1w - beta_7*X_1h -gamma_1)) * \
        (1-logit_choic_pr(-beta_2 * X_2a - beta_4*X_2e - beta_6*X_2w - beta_8*X_2h-gamma_2))

        # basically simplifies the likelihood function to three cases (note that they do not intersect)
        if (d_1 == 0) & (d_2 == 0):
            logL += np.log(P_00)
        elif (d_1 == 1) & (d_2 == 1):
            logL += np.log(P_11)
        else: 
            logL += np.log(1-P_00-P_11)
    return -logL

In [33]:
%%time
# Create a global variable to track the number of optimizer iterations
iteration_count = 0

# Callback function to update the current optimizer iteration number on the same line
def optimizer_callback(x):
    global iteration_count
    iteration_count += 1
    sys.stdout.write(f"\rCurrent optimizer iteration: {iteration_count};   ")
    sys.stdout.flush()

# Run the optimization and show the current iteration number on the same line
est_neg = minimize(MaxL_negative, 
               x0=[0.1 , -0.1, 0.08, 0.08, -0.05, -0.05, 0.06, 0.06, -0.03, -0.02], 
               callback=optimizer_callback).x

Current optimizer iteration: 49;   CPU times: total: 15min 35s
Wall time: 22min 28s


Start preparing the latex table

In [34]:
# list of columns to remove
remove_col = ["HH_ID", "drink1_husband", "drink2_wife"]
extra_col = ["Assumed effect sign"]
table_columns = extra_col +\
["drink1_husband", "drink2_wife"] + [col for col in data2.columns if col not in remove_col]

neg_peer_effects_table = pd.DataFrame([["Negative"] + est_neg.tolist()],\
                                      columns=table_columns)
neg_peer_effects_table

,Assumed effect sign,drink1_husband,drink2_wife,educ_husband,age1_husband,weight1_husband,height1_husband,educ_wife,age2_wife,weight2_wife,height2_wife
0,Negative,0.454783,2.436609,0.0248,0.031702,-0.00139,0.014324,-0.014541,0.005091,0.004868,-0.006652


#### Positive peer effects case

Basically the same code, just another probalities

In [35]:
def MaxL_positive(param):
    gamma_1, gamma_2, beta_1, beta_2 ,beta_3, beta_4, beta_5, beta_6, beta_7, beta_8 = param
    logL = 0 # as before - camulative log-likelihood
    for i in range(data2.shape[0]): # working with the initial dataset
        # define outcome variables
        d_1 = data2['drink1_husband'][i]
        d_2 = data2['drink2_wife'][i]
        
        # define covariates
        # age
        X_1a = data2['age1_husband'][i]
        X_2a = data2['age2_wife'][i]
        #educ
        X_1e = data2['educ_husband'][i]
        X_2e = data2['educ_wife'][i]
        # weight
        X_1w = data2['weight1_husband'][i]
        X_2w = data2['weight2_wife'][i]
        # height
        X_1h = data2['height1_husband'][i]
        X_2h = data2['height2_wife'][i]

        # define probabilities     
        P_01 = logit_choic_pr(-beta_1 * X_1a - beta_3*X_1e - beta_5*X_1w - beta_7*X_1h-gamma_1) *\
        (1-logit_choic_pr(-beta_2 * X_2a - beta_4*X_2e - beta_6*X_2w - beta_8*X_2h))
        
        P_10 = (1-logit_choic_pr(-beta_1 * X_1a - beta_3*X_1e - beta_5*X_1w - beta_7*X_1h)) *\
        logit_choic_pr(-beta_2 * X_2a - beta_4*X_2e - beta_6*X_2w - beta_8*X_2h-gamma_2)
        
        if (d_1 == 0) & (d_2 == 1):
            logL += np.log(P_01)
        elif (d_1 == 1) & (d_2 == 0):
            logL += np.log(P_10)
        else: 
            logL += np.log(1-P_01-P_10)
    return -logL

In [36]:
%%time
# Create a global variable to track the number of optimizer iterations
iteration_count = 0

# Callback function to update the current optimizer iteration number on the same line
def optimizer_callback(x):
    global iteration_count
    iteration_count += 1
    sys.stdout.write(f"\rCurrent optimizer iteration: {iteration_count};   ")
    sys.stdout.flush()

# Run the optimization and show the current iteration number on the same line
est_pos = minimize(MaxL_positive, 
               x0=[-0.1 , 0.1, 0.08, 0.08, -0.05, -0.05, 0.06, 0.06, -0.03, -0.02], 
               callback=optimizer_callback).x

Current optimizer iteration: 50;   CPU times: total: 19min 39s
Wall time: 28min 9s


Get the final latex table

In [37]:
pos_peer_effects_table = pd.DataFrame([["Positive"] + est_pos.tolist()],\
                                      columns=table_columns)

peer_table_final= pd.concat([neg_peer_effects_table, pos_peer_effects_table], ignore_index=True)

# Convert to LaTeX format with 3 decimal places using float_format
latex_output_3 = peer_table_final.to_latex(float_format="%.3f", index=False)

# Save to a .tex file
with open("2_2_1.tex", "w") as f:
    f.write(latex_output_3)


In [38]:
peer_table_final

,Assumed effect sign,drink1_husband,drink2_wife,educ_husband,age1_husband,weight1_husband,height1_husband,educ_wife,age2_wife,weight2_wife,height2_wife
0,Negative,0.454783,2.436609,0.024800,0.031702,-0.001390,0.014324,-0.014541,0.005091,0.004868,-0.006652
1,Positive,1.922926,2.012935,0.066069,0.031829,-0.038127,-0.030026,0.015409,0.006971,-0.016703,-0.012964


Contraiction - for the model with assumed negative peer effects (drink1_husband, drink2_wife)
got positive estimates of peer effects => do not believe this model specification.
Thus, calculate conditional probabilities only for the case of the model without contradiction - positive peer effects.

#### Conditional probabilities (positive peer effects)
Estimate, using estimated beta and gamma parameters.

In [39]:
# get estimated betas
gamma_1, gamma_2, beta_1, beta_2 ,beta_3, beta_4, beta_5, beta_6, beta_7, beta_8 = est_pos

# define outcome variables from data
d_1 = np.array(data2['drink1_husband'])
d_2 = np.array(data2['drink2_wife'])

# use covariates from the data
# age
X_1a = np.array(data2['age1_husband'])
X_2a = np.array(data2['age2_wife'])
#educ
X_1e = np.array(data2['educ_husband'])
X_2e = np.array(data2['educ_wife'])
# weight
X_1w = np.array(data2['weight1_husband'])
X_2w = np.array(data2['weight2_wife'])
# height
X_1h = np.array(data2['height1_husband'])
X_2h = np.array(data2['height2_wife'])

In [40]:
# calculate all conditional probabilities

P_01 = logit_choic_pr(-beta_1 * X_1a - beta_3*X_1e - beta_5*X_1w - beta_7*X_1h-gamma_1) *\
(1-logit_choic_pr(-beta_2 * X_2a - beta_4*X_2e - beta_6*X_2w - beta_8*X_2h))

P_10 = (1-logit_choic_pr(-beta_1 * X_1a - beta_3*X_1e - beta_5*X_1w - beta_7*X_1h)) *\
logit_choic_pr(-beta_2 * X_2a - beta_4*X_2e - beta_6*X_2w - beta_8*X_2h-gamma_2)

P_00_u = logit_choic_pr(-beta_1 * X_1a - beta_3*X_1e - beta_5*X_1w - beta_7*X_1h) *\
logit_choic_pr(-beta_2 * X_2a - beta_4*X_2e - beta_6*X_2w - beta_8*X_2h)

P_00_l = P_00_u - (logit_choic_pr(-beta_1 * X_1a - beta_3*X_1e - beta_5*X_1w - beta_7*X_1h)\
                   - logit_choic_pr(-beta_1 * X_1a - beta_3*X_1e - beta_5*X_1w - beta_7*X_1h-gamma_1))*\
(logit_choic_pr(-beta_2 * X_2a - beta_4*X_2e - beta_6*X_2w - beta_8*X_2h)\
 - logit_choic_pr(-beta_2 * X_2a - beta_4*X_2e - beta_6*X_2w - beta_8*X_2h - gamma_2))

P_11_u = 1 - P_01 - P_10 - P_00_l

P_11_l = 1 - P_01 - P_10 - P_00_u

Now into latex

In [41]:
cond_prob = pd.DataFrame({
    'P_01': P_01,
    'P_10': P_10,
    'P_00_l': P_10,
    'P_00_u': P_00_u,
    'P_11_l': P_11_l,
    'P_11_u': P_11_l
})

# Convert to LaTeX format with 3 decimal places using float_format
latex_output_4 = cond_prob.describe().to_latex(float_format="%.3f")

# Save to a .tex file
with open("2_2_2.tex", "w") as f:
    f.write(latex_output_4)

In [42]:
cond_prob.head(5)

,P_01,P_10,P_00_l,P_00_u,P_11_l,P_11_u
0,0.044805,0.111808,0.111808,0.618197,0.225190,0.225190
1,0.023183,0.143326,0.143326,0.234844,0.598646,0.598646
2,0.048648,0.108967,0.108967,0.546445,0.295941,0.295941
3,0.041463,0.108577,0.108577,0.337541,0.512420,0.512420
4,0.048636,0.108782,0.108782,0.553802,0.288781,0.288781


In [43]:
cond_prob.describe()

,P_01,P_10,P_00_l,P_00_u,P_11_l,P_11_u
count,13945.000000,13945.000000,13945.000000,13945.000000,13945.000000,13945.000000
mean,0.035386,0.137544,0.137544,0.389995,0.437075,0.437075
std,0.016782,0.033819,0.033819,0.122960,0.118804,0.118804
min,0.003719,0.031354,0.031354,0.057550,0.163507,0.163507
25%,0.024706,0.118574,0.118574,0.293040,0.342224,0.342224
50%,0.032407,0.137692,0.137692,0.392462,0.429718,0.429718
75%,0.041638,0.156327,0.156327,0.487770,0.525031,0.525031
max,0.154775,0.343492,0.343492,0.699162,0.793157,0.793157
